In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

## Load data function

In [4]:
def read_csv_file(stock, date):
    "This script loads intraday stock data CSVs for a given date. "
    "It determines the most recent Thursday (before or after the given date), "
    "constructs possible file paths using that Thursday, and tries to read the data. "
    "If found, the CSV is loaded into a DataFrame; otherwise, it raises an error."
    
    dt = datetime.strptime(date, "%Y%m%d")
    year = dt.year
    month = dt.month

    def get_last_thursday(y, m):
        # Get last day of the month
        if m == 12:
            last_day = datetime(y + 1, 1, 1) - timedelta(days=1)
        else:
            last_day = datetime(y, m + 1, 1) - timedelta(days=1)
        offset = (last_day.weekday() - 3) % 7
        return last_day - timedelta(days=offset)

    # Determine relevant last Thursday
    last_thurs_current = get_last_thursday(year, month)

    if last_thurs_current < dt:
        # If current month's last Thursday is before the given date, go to next month
        if month == 12:
            next_month = 1
            next_year = year + 1
        else:
            next_month = month + 1
            next_year = year
        last_thurs = get_last_thursday(next_year, next_month)
    else:
        last_thurs = last_thurs_current

    last_thurs_str = last_thurs.strftime("%Y%m%d")

    file_paths = [
        f"{stock}/{year}/{year}_new/{stock}_{date}_{last_thurs_str}_Intraday_Preprocessed.csv",  
        f"{stock}/{year}/{stock}_{date}_{last_thurs_str}_Intraday_Preprocessed.csv",            
        f"{stock}/{year}_new/{stock}_{date}_{last_thurs_str}_Intraday_Preprocessed.csv",        
    ]

    for file_path in file_paths:
        if os.path.exists(file_path):
            return pd.read_csv(file_path)

    print("File not found in any of the expected locations:")
    for path in file_paths:
        print("  " + path)

    return None

def load_stock_data(stocks, date):
    data = []
    for stock in stocks:
        df = read_csv_file(stock, date)
        if df is None:
            raise FileNotFoundError(f"Data missing for stock: {stock}")
        data.append(df)
    return data

## Realistic backtest (with ask and bid)

In [61]:
def simulate_hedged_straddle_day(
    dfs,                  # list of DataFrames for each stock
    stocks,
    weights,
    start_date,
    hedge_interval_minutes,
    lot_sizes,            # list of lot sizes per stock, e.g. [75, 125]
    strategy="H1",        # "H1" for Single Stock Approach, "H2" for Portfolio Approach
    r=0.06,
    dividend_yield=0.0,
    cash=1000000,
    transaction_cost_per_unit=0.0,  # cost per option contract traded
    option_transaction_cost=0.0     # cost per option contract for straddles
):
    "This function simulates a hedged straddle trading strategy for a single trading day."
    "It buys ATM straddles for specified stocks, hedges the delta exposure at regular intervals based on either"
    "H1 : individual stock or H2 : portfolio-level strategy,"
    "and unwinds all positions at the end of the day."
    "It tracks portfolio value, PnL, and handles realistic execution with bid-ask spreads and transaction costs."

    # --- Step: Preprocess and Combine Input DataFrames ---
 
    processed_dfs = []
    for df in dfs:
        # Standardize timestamp column name
        df = df.rename(columns={'Date Time': 'Timestamp'})
        # Convert 'Timestamp' column to datetime
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        # Add 'Stock' column from 'Instrument' if it exists
        if 'Instrument' in df.columns:
            df['Stock'] = df['Instrument']
        else:
            # Ensure required identifier column exists
            raise ValueError("Each input df must have 'Instrument' column for stock name")
        processed_dfs.append(df)

    # Concatenate all individual stock DataFrames into a single DataFrame
    df_all_stocks = pd.concat(processed_dfs, ignore_index=True)

    # Set a multi-level index for easier access: Timestamp, Stock, Strike, and Option Type (CE/PE)
    index_cols = ['Timestamp', 'Stock', 'Strike', 'Type']
    df_all_stocks.set_index(index_cols, inplace=True)
    df_all_stocks.sort_index(inplace=True)

    df = df_all_stocks[df_all_stocks.index.get_level_values('Timestamp').date == pd.to_datetime(start_date).date()]

    # --- Hedge times ---
    start_time = pd.to_datetime(str(start_date) + ' 09:20:00')
    end_time = pd.to_datetime(str(start_date) + ' 15:15:00')
    hedge_times = pd.date_range(start=start_time, end=end_time, freq=f'{hedge_interval_minutes}min')

    # Helper to find closest timestamp in df for a given target_time
    def get_closest_timestamp(target_time):
        timestamps = df.index.get_level_values('Timestamp').unique()
        if len(timestamps) == 0:
            raise ValueError("No timestamps available in dataframe.")
        if target_time <= timestamps[0]:
            return timestamps[0]
        if target_time >= timestamps[-1]:
            return timestamps[-1]
        idx = timestamps.get_indexer([target_time], method='ffill')[0]
        return timestamps[idx]

    # Initialize portfolio - key addition: track straddle positions
    if strategy == "H1":
        hedge_positions = {stock: 0.0 for stock in stocks}  # Individual stock positions
    elif strategy == "H2":
        hedge_positions = {stock: 0.0 for stock in stocks}  # Portfolio hedge distributed across stocks
    else:
        raise ValueError("Strategy must be 'H1' or 'H2'")
        
    straddle_positions = {stock: {'quantity': 0, 'strike': 0, 'entry_price': 0} for stock in stocks}
    cash = float(cash)
    pnl_records = []
    initial_cash = cash

    if len(lot_sizes) != len(stocks):
        raise ValueError("lot_sizes length must match stocks length")

    # STEP 1: Initial straddle purchase at first hedge time
    first_time = hedge_times[0]
    actual_first_time = get_closest_timestamp(first_time)
    
    for i, (stock, weight) in enumerate(zip(stocks, weights)):
        lot = lot_sizes[i]
        try:
            df_stock = df.xs(stock, level='Stock')
            spot = df_stock.xs(actual_first_time, level='Timestamp')['Spot'].iloc[0]
            strikes = df_stock.index.get_level_values('Strike').unique()
            atm_strike = strikes[np.abs(strikes - spot).argmin()]

            # Get initial straddle prices - BUY at ASK price
            ce_ask_price = float(df.loc[(actual_first_time, stock, atm_strike, 'CE'), 'AskPrice'])
            pe_ask_price = float(df.loc[(actual_first_time, stock, atm_strike, 'PE'), 'AskPrice'])
            
            # Calculate total capital to allocate for this stock
            capital_allocated = initial_cash * weight

            # Estimate cost of one lot of straddles
            straddle_price = ce_ask_price + pe_ask_price
            lot_cost = straddle_price * lot

            # Compute how many full lots we can afford within allocated capital
            num_lots = int(capital_allocated // lot_cost)

            # Final quantity (must be multiple of lot size)
            straddle_quantity = num_lots * lot

            if straddle_quantity == 0:
                print(f"Insufficient capital to buy even one lot of {stock} straddle.")
                continue

            # Total cost
            straddle_cost = straddle_price * straddle_quantity

            # Buy straddle
            cash -= straddle_cost
            cash -= straddle_quantity * option_transaction_cost  # Transaction cost

            # Record position
            straddle_positions[stock] = {
                'quantity': straddle_quantity,
                'strike': atm_strike,
                'entry_price': straddle_price
            }

            print(f"Initial: Bought {straddle_quantity} {stock} straddles at strike {atm_strike} for {straddle_cost:.2f}")

        except Exception as e:
            print(f"Error buying initial straddle for {stock}: {e}")
            continue

    # STEP 2: Regular hedging loop
    for current_time in hedge_times[1:-1]:  # Skip first time since we already bought straddles
        actual_time = get_closest_timestamp(current_time)
        option_values = []
        total_deltas = []

        for i, (stock, weight) in enumerate(zip(stocks, weights)):
            lot = lot_sizes[i]
            try:
                if straddle_positions[stock]['quantity'] == 0:
                    option_values.append(0.0)
                    total_deltas.append(0.0)
                    continue
                    
                strike = straddle_positions[stock]['strike']
                quantity = straddle_positions[stock]['quantity']

                # Get current option prices for valuation (use mid_price for mark-to-market)
                ce_mid_price = float(df.loc[(actual_time, stock, strike, 'CE'), 'mid_price'])
                pe_mid_price = float(df.loc[(actual_time, stock, strike, 'PE'), 'mid_price'])
                ce_delta = float(df.loc[(actual_time, stock, strike, 'CE'), 'Delta'])
                pe_delta = float(df.loc[(actual_time, stock, strike, 'PE'), 'Delta'])

                # Current value of our straddle position (mark-to-market at mid)
                straddle_value = (ce_mid_price + pe_mid_price) * quantity
                
                # Total delta for our position
                total_delta = (ce_delta + pe_delta) * quantity

                option_values.append(straddle_value)
                total_deltas.append(total_delta)
                
            except Exception as e:
                print(f"Error processing {stock} at {actual_time}: {e}")
                option_values.append(0.0)
                total_deltas.append(0.0)
                continue

        # Hedge adjustment based on strategy
        if strategy == "H1":
            # H1: Single Stock Approach - hedge each stock individually
            for i, (stock, weight) in enumerate(zip(stocks, weights)):
                if straddle_positions[stock]['quantity'] == 0:
                    continue
                    
                old_pos = hedge_positions[stock]
                required_hedge = -total_deltas[i]  # -w_j*d^t_j for stock j
                trade_qty = required_hedge - old_pos
                
                if abs(trade_qty) > 0.01:  # Only trade if meaningful change
                    trade_cost = abs(trade_qty) * transaction_cost_per_unit
                    cash -= trade_cost

                    try:
                        spot_price = df.xs((actual_time, stock), level=('Timestamp', 'Stock'))['Spot'].iloc[0]
                        
                        # Use bid/ask for realistic execution
                        if trade_qty > 0:  # Buying stock - pay ask price
                            # Assumption spot price is mid, add half spread
                            execution_price = spot_price * 1.0005  # Approximate 0.05% spread
                        else:  # Selling stock - receive bid price
                            execution_price = spot_price * 0.9995
                            
                        cash -= trade_qty * execution_price
                        hedge_positions[stock] = required_hedge
                    except Exception as e:
                        print(f"Error hedging {stock}: {e}")
                        
        elif strategy == "H2":
            # H2: Portfolio Approach - calculate total portfolio delta and distribute hedge
            total_portfolio_delta = sum(total_deltas)  # OD^t = sum of all deltas
            
            if abs(total_portfolio_delta) > 0.01:  # Only hedge if meaningful portfolio delta
                # Calculate total portfolio hedge needed: -OD^t * sum(w_i * F_i)
                # Since we're using spot prices, we distribute the hedge proportionally by weight
                total_portfolio_hedge = -total_portfolio_delta
                
                # Get current total hedge position
                current_total_hedge = sum(hedge_positions.values())
                total_trade_qty = total_portfolio_hedge - current_total_hedge
                
                if abs(total_trade_qty) > 0.0:
                    # Distribute the portfolio hedge proportionally across stocks by weight
                    for i, (stock, weight) in enumerate(zip(stocks, weights)):
                        if straddle_positions[stock]['quantity'] == 0:
                            continue
                            
                        # Calculate this stock's portion of the portfolio hedge
                        stock_hedge_portion = total_portfolio_hedge * weight
                        old_pos = hedge_positions[stock]
                        trade_qty = stock_hedge_portion - old_pos
                        
                        if abs(trade_qty) > 0.01:
                            trade_cost = abs(trade_qty) * transaction_cost_per_unit
                            cash -= trade_cost

                            try:
                                spot_price = df.xs((actual_time, stock), level=('Timestamp', 'Stock'))['Spot'].iloc[0]
                                
                                # Use bid/ask for realistic execution
                                if trade_qty > 0:  # Buying stock - pay ask price
                                    execution_price = spot_price * 1.0005  # Approximate 0.05% spread
                                else:  # Selling stock - receive bid price
                                    execution_price = spot_price * 0.9995
                                    
                                cash -= trade_qty * execution_price
                                hedge_positions[stock] = stock_hedge_portion
                            except Exception as e:
                                print(f"Error hedging {stock} in H2: {e}")

        # Calculate portfolio values (use mid prices for mark-to-market)
        option_value_total = sum(option_values)
        hedge_value = 0.0
        
        for stock in stocks:
            if hedge_positions[stock] != 0:
                try:
                    # Use mid price (spot) for mark-to-market valuation of hedge positions
                    spot_price = df.xs((actual_time, stock), level=('Timestamp', 'Stock'))['Spot'].iloc[0]
                    hedge_value += hedge_positions[stock] * spot_price
                except:
                    pass

        total_portfolio_value = option_value_total + hedge_value + cash

        pnl_records.append({
            "Timestamp": current_time,
            "OptionValue": option_value_total,
            "HedgeValue": hedge_value,
            "Cash": cash,
            "TotalValue": total_portfolio_value,
            "PnL": total_portfolio_value - initial_cash,
            "Strategy": strategy,
            "TotalPortfolioDelta": sum(total_deltas) if strategy == "H2" else None
        })

    # STEP 3: End-of-day unwind - sell all positions
    final_time = hedge_times[-1]
    actual_final_time = get_closest_timestamp(final_time)
    total_itm_loss = 0.0

    for i, stock in enumerate(stocks):
        if straddle_positions[stock]['quantity'] == 0:
            continue
            
        quantity = straddle_positions[stock]['quantity']
        strike = straddle_positions[stock]['strike']
        
        try:
            # Get final spot price
            spot = df.xs((actual_final_time, stock), level=('Timestamp', 'Stock'))['Spot'].iloc[0]
            
            # Get final option prices - SELL at BID price
            ce_bid_price = float(df.loc[(actual_final_time, stock, strike, 'CE'), 'BidPrice'])
            pe_bid_price = float(df.loc[(actual_final_time, stock, strike, 'PE'), 'BidPrice'])
            
            # Sell the straddles at bid prices
            straddle_proceeds = (ce_bid_price + pe_bid_price) * quantity
            cash += straddle_proceeds
            cash -= quantity * option_transaction_cost  # Transaction cost for selling
            
            # Calculate ITM loss (intrinsic value lost)
            if spot > strike:
                # Call is ITM
                itm_value = max(0, spot - strike) * quantity
                total_itm_loss += itm_value
            elif spot < strike:
                # Put is ITM
                itm_value = max(0, strike - spot) * quantity
                total_itm_loss += itm_value
            
            print(f"Final: Sold {quantity} {stock} straddles for {straddle_proceeds:.2f}, ITM loss: {itm_value if spot != strike else 0:.2f}")
            
        except Exception as e:
            print(f"Error unwinding {stock}: {e}")

        # Close hedge position
        old_hedge_pos = hedge_positions[stock]
        if abs(old_hedge_pos) > 0.01:
            try:
                spot_price = df.xs((actual_final_time, stock), level=('Timestamp', 'Stock'))['Spot'].iloc[0]
                
                # Use bid/ask for realistic execution when closing hedge
                if old_hedge_pos > 0:  # Selling long stock position - receive bid price
                    execution_price = spot_price * 0.9995
                else:  # Covering short stock position - pay ask price
                    execution_price = spot_price * 1.0005
                    
                cash += old_hedge_pos * execution_price
                hedge_positions[stock] = 0.0
            except Exception as e:
                print(f"Error closing hedge for {stock}: {e}")

    final_portfolio_value = cash
    total_pnl = final_portfolio_value - initial_cash

    pnl_df = pd.DataFrame(pnl_records)
    final_report = {
        "initial_cash": initial_cash,
        "final_cash": cash,
        "total_pnl": total_pnl,
        "total_itm_loss": total_itm_loss,
        "strategy_used": strategy
    }

    return pnl_df, final_portfolio_value, final_report

In [62]:
stocks = ["TATAMOTORS", "TCS", "TITAN"] # Available "ASIANPAINT", "BAJAJ-AUTO", "BAJFINANCE", "HDFCBANK", "ICICIBANK", "RELIANCE", "SBIN", "TATAMOTORS", "TCS", "TITAN"
weight = [1/len(stocks)] * len(stocks) # weights for the assets
start_date = "20240101"

# Lot_sizes
df = pd.read_csv('NIFTY_200_Lot_Size.csv')
df['Symbol'] = df['Symbol'].str.strip().str.upper()
df['LOT_SIZE'] = pd.to_numeric(df['LOT_SIZE'], errors='coerce')

lot_sizes = []

for symbol in stocks:
    symbol = symbol.strip().upper()
    lot_size_row = df[df['Symbol'] == symbol]

    if not lot_size_row.empty:
        lot_size = lot_size_row['LOT_SIZE'].values[0]
    else:
        lot_size = None

    lot_sizes.append(lot_size)

In [69]:
pnl_df, total_value, final_report = simulate_hedged_straddle_day(
    load_stock_data(stocks, start_date),  
    stocks=stocks,
    weights=weight,
    start_date=start_date,
    hedge_interval_minutes=1,
    lot_sizes=lot_sizes,
    strategy="H2",
    r=0.06,
    dividend_yield=0.0,
    cash=1000000,
    transaction_cost_per_unit=0.0,
    option_transaction_cost=0.0
)

Initial: Bought 6050.0 TATAMOTORS straddles at strike 795.0 for 331540.00
Initial: Bought 1400.0 TCS straddles at strike 3800.0 for 301070.00
Initial: Bought 1575.0 TITAN straddles at strike 3700.0 for 300037.50
Final: Sold 6050.0 TATAMOTORS straddles for 310970.00, ITM loss: 1996.50
Final: Sold 1400.0 TCS straddles for 295750.00, ITM loss: 44800.00
Final: Sold 1575.0 TITAN straddles for 283027.50, ITM loss: 8930.25


In [70]:
print("Final Total Portfolio Value:", total_value)
print("\nFinal Report:")
for key, value in final_report.items():
    print(f"{key}: {value}")

print(pnl_df.tail())

Final Total Portfolio Value: 938435.331572972

Final Report:
initial_cash: 1000000.0
final_cash: 938435.331572972
total_pnl: -61564.668427027995
total_itm_loss: 55726.750000000364
strategy_used: H2
              Timestamp  OptionValue    HedgeValue          Cash  \
349 2024-01-01 15:10:00   894204.375 -2.023270e+06  2.068950e+06   
350 2024-01-01 15:11:00   892302.500 -1.671611e+06  1.719943e+06   
351 2024-01-01 15:12:00   891655.000 -1.642430e+06  1.691216e+06   
352 2024-01-01 15:13:00   893443.750 -1.659886e+06  1.709320e+06   
353 2024-01-01 15:14:00   892978.750 -1.513115e+06  1.562739e+06   

        TotalValue           PnL Strategy  TotalPortfolioDelta  
349  939884.205227 -60115.794773       H2           726.865092  
350  940635.419380 -59364.580620       H2           601.370868  
351  940440.627525 -59559.372475       H2           591.038280  
352  942878.343466 -57121.656534       H2           597.559091  
353  942603.015583 -57396.984417       H2           544.807918  


In [71]:
pnl_df.tail()

,Timestamp,OptionValue,HedgeValue,Cash,TotalValue,PnL,Strategy,TotalPortfolioDelta
349,2024-01-01 15:10:00,894204.375,-2.023270e+06,2.068950e+06,939884.205227,-60115.794773,H2,726.865092
350,2024-01-01 15:11:00,892302.500,-1.671611e+06,1.719943e+06,940635.419380,-59364.580620,H2,601.370868
351,2024-01-01 15:12:00,891655.000,-1.642430e+06,1.691216e+06,940440.627525,-59559.372475,H2,591.038280
352,2024-01-01 15:13:00,893443.750,-1.659886e+06,1.709320e+06,942878.343466,-57121.656534,H2,597.559091
353,2024-01-01 15:14:00,892978.750,-1.513115e+06,1.562739e+06,942603.015583,-57396.984417,H2,544.807918
